In [1]:
import os
import pdb
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import random
import ipdb

print(f'Can I can use GPU now? -- {torch.cuda.is_available()}')


Can I can use GPU now? -- True


Load data from the KITTI dataset and perform train-test split:


In [17]:
class KITTIDataset(Dataset):
    def __init__(self, root_dir):
        """
        Args:
            root_dir (string): Directory with all the point clouds.
        """
        self.root_dir = root_dir
        self.files = [f for f in os.listdir(root_dir) if os.path.isfile(os.path.join(root_dir, f))]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        point_cloud_path = os.path.join(self.root_dir, self.files[idx])
        point_cloud = self.load_point_cloud_from_bin(point_cloud_path)
        return point_cloud
    
    def load_point_cloud_from_bin(self, bin_path):
        with open(bin_path, 'rb') as f:
            content = f.read()
            point_cloud = np.frombuffer(content, dtype=np.float32)
            point_cloud = point_cloud.reshape(-1, 4)  # KITTI point clouds are (x, y, z, intensity)
        return torch.from_numpy(point_cloud)
        

train_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/velodyne_reduced'
test_dir = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/testing/velodyne_reduced'

train_set = KITTIDataset(root_dir=train_dir)
test_set = KITTIDataset(root_dir=test_dir)
        
        
#batched_train_set = DataLoader(train_set, batch_size=4, shuffle=False)
#batched_test_set = DataLoader(test_set, batch_size=4, shuffle=False)


How are we getting labels?

In [26]:
label_path = '/home/adlink/Documents/ECE-57000/ClassProject/Candidate2/PointPillars/dataset/kitti/training/label_2'
label_files = [f for f in os.listdir(label_path) if os.path.isfile(os.path.join(label_path, f))]

label_path = os.path.join(label_path, label_files[0]) # A single sample for now:
with open(label_path, 'rb') as f:
    content = f.read()
    
str(content)


"b'Car 0.00 1 2.88 510.73 157.13 746.01 251.35 1.46 1.53 3.80 0.32 1.09 12.29 2.90\\nCar 0.00 0 -3.02 195.37 146.41 471.42 252.08 1.53 1.40 3.94 -4.24 1.02 11.45 2.92\\nCar 0.78 0 -2.60 0.00 148.79 85.45 256.99 1.48 1.55 3.29 -9.75 1.06 10.14 2.93\\nCyclist 0.00 0 2.35 585.64 152.87 724.54 307.90 1.57 0.52 1.66 0.56 1.29 7.92 2.42\\n'"

In [24]:
content.split('\\n')

TypeError: a bytes-like object is required, not 'str'

['007147.txt',
 '007418.txt',
 '002750.txt',
 '004214.txt',
 '002831.txt',
 '004302.txt',
 '001829.txt',
 '006076.txt',
 '005841.txt',
 '005734.txt',
 '000476.txt',
 '006701.txt',
 '004271.txt',
 '005051.txt',
 '006779.txt',
 '003250.txt',
 '003466.txt',
 '000500.txt',
 '000592.txt',
 '005167.txt',
 '003350.txt',
 '004029.txt',
 '001240.txt',
 '004919.txt',
 '001345.txt',
 '006526.txt',
 '004477.txt',
 '005115.txt',
 '004017.txt',
 '003114.txt',
 '006055.txt',
 '004412.txt',
 '004207.txt',
 '007397.txt',
 '004108.txt',
 '003306.txt',
 '004579.txt',
 '003919.txt',
 '001387.txt',
 '002797.txt',
 '005019.txt',
 '002459.txt',
 '002400.txt',
 '006512.txt',
 '004462.txt',
 '006627.txt',
 '001816.txt',
 '000808.txt',
 '007004.txt',
 '006531.txt',
 '006399.txt',
 '003471.txt',
 '001349.txt',
 '006179.txt',
 '006415.txt',
 '005731.txt',
 '001330.txt',
 '006705.txt',
 '000960.txt',
 '005791.txt',
 '000565.txt',
 '003263.txt',
 '005074.txt',
 '004998.txt',
 '003305.txt',
 '001616.txt',
 '006262.t

In [18]:
# IMPORTANT: Set to CPU for pillarization or GPU memory might get full
device =  torch.device('cpu')

class PillarFeatureNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(PillarFeatureNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        
        self.to(device)

    def forward(self, x):
        # Input x is of shape (D, P, N)
        # Convert it to (P, D, N) for 1x1 convolution      
        x = x.to(device)
        x = x.permute(1, 0, 2)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        ipdb.set_trace()
        # Max pooling operation over the points' dimension
        x, _ = torch.max(x, dim=2)  # Output shape: (P, C)
        return x.T  # Output shape: (C, P)



class Pillarization:
    def __init__(self, x_min, x_max, y_min, y_max, z_min, z_max, pillar_size, max_points_per_pillar, aug_dim):
        self.x_min = x_min
        self.x_max = x_max
        self.y_min = y_min
        self.y_max = y_max
        self.z_min = z_min
        self.z_max = z_max
        self.pillar_size = pillar_size
        self.max_points_per_pillar = max_points_per_pillar
        self.aug_dim = aug_dim
        self.num_x_pillars = int((self.x_max - self.x_min) / self.pillar_size[0])
        self.num_y_pillars = int((self.y_max - self.y_min) / self.pillar_size[1])
        

    def make_pillars(self, points):
        """
        Convert point cloud (x, y, z) into pillars.
        """
        # Mask points outside of our defined boundaries
        
        mask = (
            (points[:, 0] >= self.x_min) & (points[:, 0] <= self.x_max) &
            (points[:, 1] >= self.y_min) & (points[:, 1] <= self.y_max) &
            (points[:, 2] >= self.z_min) & (points[:, 2] <= self.z_max)
        )
        points = points[mask]

        
        # Using numpy's digitize to find the interval/bin each point belongs to.
        self.x_indices = torch.tensor(np.digitize(points[:, 0],
            np.linspace(self.x_min, self.x_max, self.num_x_pillars))) - 1
        
        self.y_indices = torch.tensor(np.digitize(points[:, 1], 
            np.linspace(self.y_min, self.y_max, self.num_y_pillars))) - 1
        
        pillars = torch.zeros((self.num_x_pillars, self.num_y_pillars, self.max_points_per_pillar, 
            self.aug_dim))
        
        # Count how many points are in each pillar to ensure we don't exceed `max_points_per_pillar`
        count = torch.zeros((self.num_x_pillars, self.num_y_pillars), dtype=torch.long)
        
        if (device != torch.device('cpu')):
            self.x_indices.to(device)
            self.y_indices.to(device)
            points = points.to(device)
            pillars = pillars.to(device)
            count = count.to(device)
           
        # Calculate pillar x-y center:
        pillar_x_center = self.x_indices * self.pillar_size[0] + self.pillar_size[0] / 2.0
        pillar_y_center = self.y_indices * self.pillar_size[1] + self.pillar_size[1] / 2.0 

        
        # TODO: Store points in the pillars in a vectorized way filling the pillars tensor:        
        for i in range(points.shape[0]):
            x_ind = self.x_indices[i]
            y_ind = self.y_indices[i]
            
            if count[x_ind, y_ind] < self.max_points_per_pillar:
                # Compute x_c, y_c and z_c
                x_c = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0) - points[i, 0]
                y_c = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0) - points[i, 1]
                z_c = (self.z_min + self.z_max) / 2 - points[i, 2] # assuming the z-center is the midpoint
                
                # Calculate pillar center
                x_pillar_center = (x_ind * self.pillar_size[0] + self.pillar_size[0] / 2.0)
                y_pillar_center = (y_ind * self.pillar_size[1] + self.pillar_size[1] / 2.0)
                
                # Add original x, y, and z coordinates, then x_c, y_c, z_c
                pillars[x_ind, y_ind, count[x_ind, y_ind], :3] = points[i, :3]
                
                if (device != torch.device('cpu')): 
                    pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c]).to(device)
                else: 
                    pillars[x_ind, y_ind, count[x_ind, y_ind], 3:6] = torch.tensor([x_c, y_c, z_c])
                    
                pillars[x_ind, y_ind, count[x_ind, y_ind], 6] = x_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 0]
                pillars[x_ind, y_ind, count[x_ind, y_ind], 7] = y_pillar_center - pillars[x_ind, y_ind, count[x_ind, y_ind], 1]
                
                count[x_ind, y_ind] += 1
                
        
        # Zero-padding if too few point, random sampling if too many points:
        for i in range(self.num_x_pillars):
            for j in range(self.num_y_pillars):
                if pillars[i, j].shape[0] > self.max_points_per_pillar:
                    # Randomly sample points if there are too many for a given pillar
                    pillars[i, j] = pillars[i, j][torch.randperm(pillars[i, j].shape[0])[:self.max_points_per_pillar]]
                elif pillars[i, j].shape[0] < self.max_points_per_pillar:
                    # Zero pad if there are too few points for a given pillar
                    pillars[i, j] = torch.cat((pillars[i, j], torch.zeros((self.max_points_per_pillar - 
                                                                           pillars[i, j].shape[0], pillars[i, j].shape[1]))))
        
        # Reshape pillars to size (D,P,N):
        pillars = pillars.permute(3, 0, 1, 2).reshape(D, -1, N)
        
        return pillars


class PseudoImageDataset(Dataset):
    def __init__(self, train_dir, D, N, transform=None):
        self.train_dir = train_dir
        self.filenames = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]
        self.transform = transform
        
        self.pillarizer = Pillarization(aug_dim=D, x_min=-40.0, x_max=40.0, y_min=-25.0, y_max=25.0, 
                                        z_min=-3, z_max=3, pillar_size=(0.5, 0.5), max_points_per_pillar=N)
        self.feature_extractor = PillarFeatureNet(D, 64)  # TODO: Don't hardcode me

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        point_cloud = train_set.load_point_cloud_from_bin(os.path.join(self.train_dir, self.filenames[idx]))
        pillars = self.pillarizer.make_pillars(point_cloud)
        
        # Apply linear activation, batchnorm, and ReLU for feature extraction from pillars tensor
        features = self.feature_extractor(pillars)
        
        # Generate pseudo-image (GPU)
        pseudo_image = torch.zeros(features.shape[0], self.pillarizer.num_y_pillars, self.pillarizer.num_x_pillars).to(device)
        
        # Scatter the features back to their original pillar locations
        print(f'Loading point cloud number {idx}')
        for i in range(features.shape[1]):
            x_ind = self.pillarizer.x_indices[i].long() # SUS: Is the indexing here correct?
            y_ind = self.pillarizer.y_indices[i].long()
            pseudo_image[:, y_ind, x_ind] = features[:, i]

        
        if self.transform:
            pseudo_image = self.transform(pseudo_image)
            
        return pseudo_image
        
        
# Create the dataset and DataLoader
D = 9
N = 100
dataset = PseudoImageDataset(train_dir=train_dir, D=D, N=N)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


Now, get batches of data:

In [4]:
for batch_idx, (pseudo_image) in enumerate(train_loader):
    break

# After pillarization and batching, we can start using the GPU:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/tmp/ipykernel_2050086/3895991962.py:23: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  return torch.from_numpy(point_cloud)


Loading point cloud number 5995
Loading point cloud number 4978
Loading point cloud number 1309
Loading point cloud number 3359


Backbone pipeline:

In [5]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, L, stride):
        super(Block, self).__init__()
        layers = []
        # First layer with the specified stride
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1))
        layers.append(nn.BatchNorm2d(out_channels)) # TODO -> Uncomment and batch
        layers.append(nn.ReLU(inplace=True))
        
        # Subsequent layers with stride 1
        for _ in range(1, L):
            layers.append(nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1))
            layers.append(nn.BatchNorm2d(out_channels))
            layers.append(nn.ReLU(inplace=True))
        
        self.block = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.block(x)

class UpSample(nn.Module):
    def __init__(self, in_channels, out_channels, stride, output_padding):
        super(UpSample, self).__init__()
        # Assuming stride_out is always half of stride_in based on the diagram
        self.up = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, output_padding=output_padding),
            nn.BatchNorm2d(out_channels), # TODO> Uncomment and batch
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        return self.up(x)

class BackBone(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(BackBone, self).__init__()

        # Define blocks with arbitrary L for now (can be tuned based on requirements)
        self.block1 = Block(in_channels, out_channels*2, L=3, stride=1)
        self.block2 = Block(out_channels*2, out_channels*2, L=3, stride=2)
        self.block3 = Block(out_channels*2, out_channels*2, L=3, stride=2)
        

        # Define upsampling layers        
        self.up1 = UpSample(out_channels*2, out_channels*2, stride=1, output_padding=0)
        self.up2 = UpSample(out_channels*2, out_channels*2, stride=2, output_padding=1)
        self.up3 = UpSample(out_channels*2, out_channels*2, stride=4, output_padding=3)
        
    def forward(self, x):
        x1 = self.block1(x)
        x2 = self.block2(x1)
        x3 = self.block3(x2)
        x2.size()
        # Upsample and concatenate
        up_x1 = self.up1(x1)   
        up_x2 = self.up2(x2)
        up_x3 = self.up3(x3)     
        concat_features = torch.cat([up_x1, up_x2, up_x3], dim=1)
        
        return concat_features

    
backbone = BackBone(in_channels=64, out_channels=64)
backbone_features = backbone(pseudo_image)

In [6]:
backbone_features.size()




torch.Size([4, 384, 100, 160])

Now, implement the Detection Head:

In [7]:
class DetectionHead(nn.Module):
    def __init__(self, in_channels, grid_size_x, grid_size_y, num_anchors, num_classes):
        super(DetectionHead, self).__init__()
        self.grid_size_x = grid_size_x
        self.grid_size_y = grid_size_y
        self.num_anchors = num_anchors
        self.num_classes = num_classes

        # Assuming 4 anchor boxes per cell
        self.loc_layer = nn.Conv2d(in_channels, num_anchors * 3, 1)
        self.size_layer = nn.Conv2d(in_channels, num_anchors * 3, 1)
        self.clf_layer = nn.Conv2d(in_channels, num_anchors * (num_classes + 1), 1) # +1 for confidence score
        self.occupancy_layer = nn.Conv2d(in_channels, num_anchors * 1, 1)
        self.angle_layer = nn.Conv2d(in_channels, num_anchors * 1, 1)
        self.heading_layer = nn.Conv2d(in_channels, num_anchors * 1, 1)

    def forward(self, x):
        loc = self.loc_layer(x).view(x.size(0), self.num_anchors, 3, self.grid_size_x, self.grid_size_y)
        size = self.size_layer(x).view(x.size(0), self.num_anchors, 3, self.grid_size_x, self.grid_size_y)
        clf = self.clf_layer(x).view(x.size(0), self.num_anchors, self.num_classes + 1, self.grid_size_x, self.grid_size_y)
        occupancy = self.occupancy_layer(x).view(x.size(0), self.num_anchors, 1, self.grid_size_x, self.grid_size_y)
        angle = self.angle_layer(x).view(x.size(0), self.num_anchors, 1, self.grid_size_x, self.grid_size_y)
        heading = self.heading_layer(x).view(x.size(0), self.num_anchors, 1, self.grid_size_x, self.grid_size_y)

        # Adjust the shape to match the output as per the provided image. 
        #loc = loc.permute(0, 3, 4, 1, 2)
        #size = size.permute(0, 3, 4, 1, 2)
        #clf = clf.permute(0, 3, 4, 1, 2)
        #occupancy = occupancy.permute(0, 3, 4, 1, 2)
        #angle = angle.permute(0, 3, 4, 1, 2)
        #heading = heading.permute(0, 3, 4, 1, 2)

        return loc, size, clf, occupancy, angle, heading


#backbone_output = torch.randn(4, backbone_features.size()[1], 252, 252) # Example feature map from backbone
detection_head = DetectionHead(backbone_features.size()[1], backbone_features.size()[2], 
                        backbone_features.size()[3], num_anchors=1, num_classes=2) # Initialize with correct parameters
loc, size, clf, occupancy, angle, heading = detection_head(backbone_features)

In [11]:
loc.size()

torch.Size([4, 1, 3, 100, 160])

In [15]:
loc[0,:,:,:,:]

tensor([[[[-0.0429, -0.0443, -0.0088,  ...,  0.0088, -0.0071, -0.0120],
          [ 0.0092, -0.1676,  0.0165,  ..., -0.0987, -0.0149, -0.0155],
          [-0.0227, -0.0605, -0.0418,  ..., -0.0590, -0.0411, -0.0273],
          ...,
          [-0.0327, -0.1672, -0.0510,  ..., -0.0833, -0.0099, -0.0402],
          [-0.0260, -0.0372, -0.0325,  ..., -0.0378, -0.0255, -0.0422],
          [-0.0490, -0.0897, -0.0574,  ..., -0.0790, -0.0282, -0.0280]],

         [[-0.1106,  0.0586, -0.0007,  ...,  0.0246,  0.0050,  0.0606],
          [-0.0108, -0.0386,  0.0002,  ..., -0.0153,  0.0149,  0.0180],
          [-0.0167,  0.0717, -0.0153,  ...,  0.0562, -0.0230,  0.0447],
          ...,
          [ 0.0021, -0.0744,  0.0250,  ..., -0.0310,  0.0309, -0.0089],
          [ 0.0029,  0.0765,  0.0075,  ...,  0.0653, -0.0212,  0.0456],
          [ 0.0725, -0.0052,  0.0326,  ...,  0.0029,  0.0207,  0.0124]],

         [[ 0.0831,  0.0483,  0.0100,  ...,  0.0428,  0.0171, -0.0227],
          [ 0.0520,  0.0444, -